<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/47_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%capture
!pip install deap tensorflow numpy scikit-learn

In [7]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

# 1. Подготовка данных
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 2. Настройка генетического алгоритма
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Параметры для оптимизации
toolbox.register("conv_layers", random.randint, 1, 3)
toolbox.register("filters", random.choice, [32, 64, 128])
toolbox.register("dense_units", random.choice, [128, 256])
toolbox.register("learning_rate", random.choice, [0.001, 0.0005, 0.0001])
toolbox.register("optimizer", random.choice, ['adam', 'sgd'])

def create_individual():
    return [
        toolbox.conv_layers(),
        toolbox.filters(),
        toolbox.dense_units(),
        float(toolbox.learning_rate()),  # Явное преобразование к float
        toolbox.optimizer()
    ]

toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# 3. Функция оценки
def evaluate(individual):
    conv_layers, filters, dense_units, lr, optimizer = individual

    # Явная проверка типа learning rate
    if not isinstance(lr, float):
        lr = float(lr)

    model = models.Sequential()
    model.add(layers.Conv2D(filters, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))

    for _ in range(conv_layers-1):
        model.add(layers.Conv2D(filters, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(dense_units, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    # Создание оптимизатора с явным указанием типа
    if optimizer == 'adam':
        opt = Adam(learning_rate=lr)
    else:
        opt = SGD(learning_rate=lr)

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        epochs=3,  # Уменьшено для ускорения
                        validation_data=(x_val, y_val),
                        verbose=0,
                        batch_size=128)

    return (history.history['val_accuracy'][-1],)

# Настройка операторов генетического алгоритма
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)

# Раздельная мутация для разных типов параметров
def custom_mutate(individual):
    # Мутация для целых чисел
    individual[0] = random.randint(1, 3)
    individual[1] = random.choice([32, 64, 128])
    individual[2] = random.choice([128, 256])

    # Мутация для learning rate
    individual[3] = random.choice([0.001, 0.0005, 0.0001])

    # Мутация для оптимизатора
    individual[4] = random.choice(['adam', 'sgd'])
    return individual,

toolbox.register("mutate", custom_mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

# 4. Запуск оптимизации
population = toolbox.population(n=10)
hof = tools.HallOfFame(3)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

result, log = algorithms.eaSimple(population, toolbox,
                                  cxpb=0.7,
                                  mutpb=0.3,
                                  ngen=3,
                                  stats=stats,
                                  halloffame=hof,
                                  verbose=True)

# 5. Результаты
best_ind = hof[0]
print("\nЛучшие гиперпараметры:")
print(f"Сверточные слои: {best_ind[0]}")
print(f"Фильтры: {best_ind[1]}")
print(f"Нейроны в Dense: {best_ind[2]}")
print(f"Learning rate: {float(best_ind[3])}")  # Явное преобразование
print(f"Оптимизатор: {best_ind[4]}")
print(f"Валидационная точность: {best_ind.fitness.values[0]:.4f}")

# 6. Финальная оценка
final_model = models.Sequential()
final_model.add(layers.Conv2D(best_ind[1], (3, 3), activation='relu', input_shape=(28, 28, 1)))
final_model.add(layers.MaxPooling2D((2, 2)))
for _ in range(best_ind[0]-1):
    final_model.add(layers.Conv2D(best_ind[1], (3, 3), activation='relu'))
    final_model.add(layers.MaxPooling2D((2, 2)))
final_model.add(layers.Flatten())
final_model.add(layers.Dense(best_ind[2], activation='relu'))
final_model.add(layers.Dense(10, activation='softmax'))

optimizer = Adam(learning_rate=float(best_ind[3])) if best_ind[4] == 'adam' else SGD(learning_rate=float(best_ind[3]))
final_model.compile(optimizer=optimizer,
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

final_model.fit(x_train, y_train, epochs=10, batch_size=128, verbose=0)
test_loss, test_acc = final_model.evaluate(x_test, y_test, verbose=0)
print(f"\nТочность на тестовых данных: {test_acc:.4f}")

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


gen	nevals	avg     	min     	max     
0  	10    	0.574292	0.177333	0.988417
1  	6     	0.824233	0.199   	0.988417
2  	7     	0.968233	0.855333	0.988417


ValueError: Argument `learning_rate` should be float, or an instance of LearningRateSchedule, or a callable (that takes in the current iteration value and returns the corresponding learning rate value). Received instead: learning_rate=1